In [1]:
import numpy as np
import matplotlib.pyplot as plt

The aim is to solve the following variational problem
$$\min_u \frac{1}{2} \|Ku - f^{\delta}\|_2^2 + \alpha \|Lu\|_1,$$
where $K$ is a given forward operator (matrix) and $L$ is a discretisation of the second derivative operator.

1. Describe and implement a standard sub-gradient-descent method for solving this variational problem.
2. Describe and implement a splitting method; y